In [42]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random

import GameManager
from GameManager import map_manager, Tile, Unit, Team

import Neat_AI
from Neat_AI import eval_genomes, run

In [63]:
from importlib import reload
reload(GameManager)
reload(Neat_AI)

from GameManager import map_manager, Tile, Unit, Team
from Neat_AI import eval_genomes, run

In [49]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config/config-feedforward')

In [65]:
Map = map_manager((4, 4))
unit1 = Map.place_unit((0,  0), 0)
pos2 = (0, 0)
while (unit1.pos == pos2):
    pos2 = (random.randint(0, Map.Map.shape[0]) - 1, random.randint(0, Map.Map.shape[1]) - 1) 
    
unit2 = Map.place_unit(pos2, 1)
print(Map)

#Map.find_movement(unit1)
#Map.move_unit(unit1, (4,1))

[0][ ][ ][ ]
[ ][ ][ ][ ]
[ ][ ][ ][ ]
[ ][ ][ ][1]



In [66]:
Map.Turn()
Map.Turn()

In [67]:
Map.find_movement(unit1)
#Map.move_unit(unit1, (4,1))
#Map.move_unit(unit1, Map.visited_tiles[random.randint(0, len(Map.visited_tiles) - 1)].pos)

input_list = []
for row in Map.Map:
    for tile in row:
        if (tile.unit_ref is not None and tile.unit_ref.Team == 1):
            input_list.append(1.0)
        else:
            input_list.append(0.0)
input_tup = tuple(input_list)
formatted_in = [ '%.2f' % elem for elem in input_tup ]
print(formatted_in)
print(" ")

output = win_net.activate(input_tup)
output_in = [ '%.2f' % elem for elem in output ]
print(output_in)
move_index = np.argmax(output) 
print((move_index // 2, move_index % 2))
Map.move_unit(unit1, (move_index // 2, move_index % 2))

['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00']
 


NameError: name 'win_net' is not defined

In [312]:
Map.print_units()

Unit:
	pos: (4, 2)
	curr_move: 4.0
	hp: 100
	max_move: 10
	Att: 20
	Def: 10
	Team: 0

Unit:
	pos: (2, 4)
	curr_move: 10
	hp: 100
	max_move: 10
	Att: 20
	Def: 10
	Team: 1



In [64]:
win_net = run(config_path)

	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn


 ****** Running generation 0 ****** 

input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['0.00', '0.02']
move: (0, 0)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['0.00', '0.02']
move: (0, 0)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['0.96', '0.99']
move: (3, 3)
Tile (3, 3) not reachable by unit
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['0.96', '0.99']
move: (3, 3)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 

move: (0, 3)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00']
output vector: ['0.00', '0.22']
move: (0, 1)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00']
output vector: ['0.00', '0.22']
move: (0, 1)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['1.00', '0.99']
move: (3, 3)
Tile (3, 3) not reachable by unit
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
output vector: ['1.00', '0.99']
move: (3, 3)
input vector: ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0

Traceback (most recent call last):
  File "C:\Users\kevin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-64-30b1abc22580>", line 1, in <module>
    win_net = run(config_path)
  File "C:\_Kevin\UT-Stuff\_Summer2023\CS370\kriek-cs370\Neat_AI.py", line 110, in run
    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
  File "C:\Users\kevin\anaconda3\lib\site-packages\neat\checkpoint.py", line 75, in restore_checkpoint
    with gzip.open(filename) as f:
  File "C:\Users\kevin\anaconda3\lib\gzip.py", line 58, in open
    binary_file = GzipFile(filename, gz_mode, compresslevel)
  File "C:\Users\kevin\anaconda3\lib\gzip.py", line 173, in __init__
    fileobj = self.myfileobj = builtins.open(filename, mode or 'rb')
FileNotFoundError: [Errno 2] No such file or directory: 'neat-checkpoint-4'

During handling of the above exception, another exception occurred:

Traceback

In [376]:
saved_net = win_net